# GPU-Jupyter

This Jupyterlab Instance is connected to the GPU via CUDA drivers. In this notebook, we test the installation and perform some basic operations on the GPU.

## Test GPU connection

#### Using the following command, your GPU type and its NVIDIA-SMI driver version should be listed:

In [1]:
!nvidia-smi

Sun Mar 24 01:33:58 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:05:00.0 Off |                  N/A |
| 29%   34C    P8     9W / 120W |      1MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### Now, test if PyTorch can access the GPU via CUDA:

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
import tensorflow as tf
print(tf.test.is_built_with_cuda(), tf.config.list_physical_devices('GPU'))

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2024-03-24 01:34:05.741623: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 01:34:05.741747: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 01:34:05.799110: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 01:34:05.917661: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-24 01:34:06.845916: W tensorflow/compiler/tf2

True [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15182812829918085901
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5291507712
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12447598201238849413
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1060 6GB, pci bus id: 0000:05:00.0, compute capability: 6.1"
xla_global_id: 416903419
]


2024-03-24 01:34:08.136888: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-24 01:34:08.142316: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-24 01:34:08.142500: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
from __future__ import print_function
import numpy as np
import torch
a = torch.rand(5, 3)
a

tensor([[0.1312, 0.0905, 0.5144],
        [0.3859, 0.0653, 0.3582],
        [0.8145, 0.5240, 0.4840],
        [0.4330, 0.4418, 0.7380],
        [0.2680, 0.3029, 0.4265]])

## Performance test

#### Now we want to know how much faster a typical operation is using GPU. Therefore we do the same operation in numpy, PyTorch and PyTorch with CUDA. The test operation is the calculation of the prediction matrix that is done in a linear regression.

### 1) Numpy

In [5]:
x = np.random.rand(10000, 256)

In [6]:
%%timeit
H = x.dot(np.linalg.inv(x.transpose().dot(x))).dot(x.transpose())

448 ms ± 4.26 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 2) PyTorch

In [7]:
x = torch.rand(10000, 256)

In [8]:
%%timeit
# Calculate the projection matrix of x on the CPU
H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

203 ms ± 1.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 3) PyTorch on GPU via CUDA

In [9]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    x = torch.rand(10000, 256, device=device) # directly create a tensor on GPU
    y = x.to(device)                       # or just use strings ``.to("cuda")``
    print(x[0:5, 0:5])
    print(y.to("cpu", torch.double)[0:5, 0:5])

tensor([[0.7506, 0.9940, 0.7261, 0.7438, 0.2055],
        [0.0778, 0.4377, 0.6793, 0.3919, 0.4553],
        [0.7606, 0.4149, 0.4151, 0.9134, 0.7298],
        [0.6129, 0.1301, 0.6732, 0.1864, 0.4667],
        [0.9432, 0.9700, 0.8249, 0.8299, 0.8453]], device='cuda:0')
tensor([[0.7506, 0.9940, 0.7261, 0.7438, 0.2055],
        [0.0778, 0.4377, 0.6793, 0.3919, 0.4553],
        [0.7606, 0.4149, 0.4151, 0.9134, 0.7298],
        [0.6129, 0.1301, 0.6732, 0.1864, 0.4667],
        [0.9432, 0.9700, 0.8249, 0.8299, 0.8453]], dtype=torch.float64)


In [10]:
%%timeit
# Calculate the projection matrix of x on the GPU
H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

19.3 ms ± 101 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Exhaustive Testing on GPU

In [16]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    x = torch.rand(10000, 10, device=device) # directly create a tensor on GPU

In [17]:
if torch.cuda.is_available():
    y = x.to(device)                       # or just use strings ``.to("cuda")``
    print(x[0:5, 0:5])

tensor([[0.1260, 0.7505, 0.5393, 0.8144, 0.9606],
        [0.3401, 0.4065, 0.9569, 0.4985, 0.3725],
        [0.9985, 0.9281, 0.2390, 0.1671, 0.4465],
        [0.5225, 0.7880, 0.4171, 0.9828, 0.1909],
        [0.6865, 0.6273, 0.3280, 0.1057, 0.3778]], device='cuda:0')


In [18]:
if torch.cuda.is_available():
    # Here is the memory of the GPU a border.
    # A matrix with 100000 lines requires 37 GB, but only 8 GB are available.
    H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

In [19]:
if torch.cuda.is_available():
    print(H[0:5, 0:5])

tensor([[ 9.7152e-04,  3.4042e-05, -3.5887e-04,  2.5985e-04, -1.8244e-04],
        [ 3.4042e-05,  8.5730e-04, -3.6825e-04, -4.4229e-05, -6.7246e-05],
        [-3.5887e-04, -3.6825e-04,  1.1322e-03,  1.2824e-05,  6.4362e-04],
        [ 2.5985e-04, -4.4229e-05,  1.2824e-05,  9.6444e-04, -6.0630e-05],
        [-1.8244e-04, -6.7246e-05,  6.4362e-04, -6.0630e-05,  4.7804e-04]],
       device='cuda:0')


In [20]:
if torch.cuda.is_available():
    # This operation is difficult, as an symmetric matrix is transferred
    # back to the CPU. Is possible up to 30000 rows.
    print(H.to("cpu", torch.double)[0:5, 0:5])

tensor([[ 9.7152e-04,  3.4042e-05, -3.5887e-04,  2.5985e-04, -1.8244e-04],
        [ 3.4042e-05,  8.5730e-04, -3.6825e-04, -4.4229e-05, -6.7246e-05],
        [-3.5887e-04, -3.6825e-04,  1.1322e-03,  1.2824e-05,  6.4362e-04],
        [ 2.5985e-04, -4.4229e-05,  1.2824e-05,  9.6444e-04, -6.0630e-05],
        [-1.8244e-04, -6.7246e-05,  6.4362e-04, -6.0630e-05,  4.7804e-04]],
       dtype=torch.float64)
